In [1]:
import generate
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pyEDM
from plotly.subplots import make_subplots

In [37]:
sigma, rho, beta = 10, 28, 8 / 3

dt = 0.01
max_t = 100

X0 = np.array([0.1, 0.1, 0.1])

t, X = generate.lorenz(sigma, rho, beta, X0, dt, max_t)

layout = go.Layout(
    title="Lorenz Attractor",
    scene=dict(xaxis=dict(title="x"), yaxis=dict(title="y"), zaxis=dict(title="z")),
    width=600,
    height=600,
)
data = go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2], mode="lines")

fig = go.Figure(data=data, layout=layout)
fig.show()

In [38]:
sample_rate = 5
tau = 2

x = X[:, 0]

down_sampled_before = np.array(
    [
        x[::sample_rate][: -2 * tau],
        x[::sample_rate][tau:-tau],
        x[::sample_rate][2 * tau :],
    ]
).T
down_sampled_after = np.array(
    [
        x[: -2 * tau],
        x[tau:-tau],
        x[2 * tau :],
    ]
)[:, ::sample_rate].T

L = min(down_sampled_before.shape[0], down_sampled_after.shape[0])
down_sampled_before = down_sampled_before[:L]
down_sampled_after = down_sampled_after[:L]

print(down_sampled_after.shape, down_sampled_before.shape)

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Sample->Embed", "Embed->Sample"),
    specs=[[{"type": "scene"}, {"type": "scene"}]],
)

fig.add_trace(
    go.Scatter3d(
        x=down_sampled_before[:, 0],
        y=down_sampled_before[:, 1],
        z=down_sampled_before[:, 2],
        mode="lines",
        showlegend=False,
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter3d(
        x=down_sampled_after[:, 0],
        y=down_sampled_after[:, 1],
        z=down_sampled_after[:, 2],
        mode="lines",
        showlegend=False,
    ),
    row=1,
    col=2,
)

fig.show()

(1996, 3) (1996, 3)


In [39]:
def plot_rho_E_tau(
    df: pd.DataFrame, columns: list[str], maxE: int, Tp: int, minTau: int
):
    EmbedDimensionKwargs = {
        "dataFrame": df,
        "lib": [1, df.shape[0] // 2],
        "pred": [df.shape[0] // 2 + 1, df.shape[0]],
        "maxE": maxE,
        "Tp": Tp,
        "showPlot": False,
    }

    tau_range = range(-1, minTau - 1, -1)
    E_range = range(1, maxE + 1)

    fig = make_subplots(
        rows=1,
        cols=len(columns),
        subplot_titles=columns,
        horizontal_spacing=0.01,
        specs=[[{"type": "surface"} for _ in range(len(columns))]],
    )

    for i, column in enumerate(columns):
        rho = []
        for tau in tau_range:
            result = pyEDM.EmbedDimension(
                columns=column, target=column, tau=tau, **EmbedDimensionKwargs
            )
            rho.append(result["rho"].values)

        E, tau = np.meshgrid(E_range, tau_range)
        rho = np.array(rho)

        fig.add_trace(
            go.Surface(x=E, y=tau, z=rho, scene=f"scene{i+1}", showscale=False),
            row=1,
            col=i + 1,
        )
    fig.update_scenes(
        xaxis_title_text="E",
        yaxis_title_text="tau",
        zaxis_title_text="rho",
        zaxis_range=[-1, 1],
    )

    fig.update_layout(
        height=400, width=len(columns) * 400, margin=dict(l=20, r=20, t=30, b=30)
    )
    fig.show()


df = pd.DataFrame(
    {
        "down_sampled_before": down_sampled_before[:, 0],
        "down_sampled_after": down_sampled_after[:, 0],
    }
)

plot_rho_E_tau(df, ["down_sampled_before", "down_sampled_after"], 5, 15, -5)